In [1]:
from ipywidgets import FloatProgress

In [1]:
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-8pgu0dws
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-8pgu0dws
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done


In [3]:
!pip install torchmetrics


In [3]:
import torch, clip, csv, json

import numpy as np
import pandas as pd
import rasterio as rio
import torch.nn.functional as F

from rasterio.enums import Resampling
from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import MultilabelAccuracy

from tqdm import tqdm

In [4]:
!nvidia-smi

Tue Jun 20 09:16:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:3B:00.0 Off |                  N/A |
|  0%   56C    P8    33W / 350W |    818MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:AF:00.0 Off |                  N/A |
|  0%   

In [4]:
root_dir = "geolibs/data/BigEarthNet-S2"
bsize, psize = 1024, 224
num_workers = 8

model_name = "ViT-L/14"

In [5]:
class BigEarthNetS1InferenceDataset(Dataset):
    def __init__(self, root_dir):
        root_vec_dir = f"{root_dir}/vectors/CSV"

        meta_file = f"{root_vec_dir}/metadata.json"
        with open(meta_file) as out:
            task_meta = json.load(out)

        classes = [lbl_meta["options"] for lbl_meta in task_meta["label:metadata"]][0]
        cls_idx_map = {cls: idx for idx, cls in enumerate(classes)}
        
        vec_df = pd.read_csv(f"{root_vec_dir}/val.csv")
        vec_df["image"] = vec_df["image:01"].apply(lambda x: f'{root_dir}/rasters/{x.split("/")[-1]}')
        vec_df["label"] = vec_df["labels"].apply(lambda x: [cls_idx_map[l] for l in x.split("\t")])
        vec_df["label"] = vec_df["label"].apply(lambda x: [1 if i in x else 0 for i in range(len(classes))])
        vec_df.drop(['image-id','image:01','date:01','type','geometry','labels'],axis=1,inplace=True)

        self.vec_df = vec_df
        self.classes = classes

    def __len__(self):
        return len(self.vec_df)

    def __getitem__(self, idx):
        df_entry = self.vec_df.loc[idx] 
        image = rio.open(df_entry["image"]).read(out_shape=[1,psize,psize], resampling=Resampling.bilinear)     
        smpl_map = {
            "image": np.concatenate([image, image, image], axis=0),
            "label": np.array(df_entry["label"], dtype=np.int16)
        }

        return smpl_map
                    

In [6]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda:0"
model, preprocess = clip.load(model_name, device=device)

In [7]:
templates = [
    'a bad photo of a {}.',
    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]

In [8]:
def zeroshot_classifier(classnames, templates):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).cuda() #tokenize
            class_embeddings = model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights

In [9]:
bens1_ds = BigEarthNetS1InferenceDataset(root_dir=root_dir)

classes = bens1_ds.classes
num_classes = len(classes)

zeroshot_weights = zeroshot_classifier(classes, templates)

FileNotFoundError: [Errno 2] No such file or directory: 'geolibs/data/BigEarthNet-S2/vectors/CSV/metadata.json'

In [11]:
from llama_cpp import Llama

In [15]:
llm = Llama(model_path="./models/7B/ggml-model.bin")

ValueError: Model path does not exist: ./models/7B/ggml-model.bin

In [13]:

bens1_dl = DataLoader(bens1_ds, batch_size=bsize, shuffle=False, num_workers=num_workers)
acc_inst = MultilabelAccuracy(num_labels=num_classes).to(device)

with torch.no_grad():
    for sample in tqdm(bens1_dl):
        images = sample["image"].to(device)
        target = sample["label"].to(device)

        im_features = model.encode_image(images)
        im_features /= im_features.norm(dim=-1, keepdim=True)
        logits = 1.* im_features @ zeroshot_weights

        acc_inf= acc_inst(logits, target)

    avg_acc_inf = acc_inst.compute()
    print(avg_acc_inf)

  0%|                                                    | 0/97 [11:27<?, ?it/s]


KeyboardInterrupt: 